In [1]:
import xgboost as xgb
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn import preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import itertools
import datetime
pd.set_option('display.max_columns', None)
from operator import itemgetter
import time
import keras
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer
import string
from textblob import TextBlob
import nltk

/Users/ishimotoyoshitake/anaconda/envs/py/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.
/Users/ishimotoyoshitake/anaconda/envs/py/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
train_data_origin = pd.read_csv("/Users/ishimotoyoshitake/Desktop/kaggle_datasets/mercari/train.tsv",delimiter='\t')
test_data_origin = pd.read_csv("/Users/ishimotoyoshitake/Desktop/kaggle_datasets/mercari/test.tsv",delimiter='\t')
concat_data = pd.concat([train_data_origin,test_data_origin],axis=0)

In [3]:
test_data_origin[380825:380828]

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
380825,380825,HUF weed socks,2,Women/Athletic Apparel/Socks,NaN,1,HUF weed socks Mint green socks With pink weed...
380826,380826,Forever 21 Crop Tank,2,"Women/Tops & Blouses/Tank, Cami",FOREVER 21,0,Button Up Mustard Color Size Small NEW Without...
380827,380827,Chanel authentic VIP bag & charms,2,Women/Women's Handbags/Totes & Shoppers,Chanel,1,New authentic Chanel VIP bag. Made of a thick ...


In [4]:
electronics_phone_list = [
    'Electronics/Cell Phones & Accessories/Cell Phone Accessories',
    'Electronics/Cell Phones & Accessories/Cell Phones & Smartphones',
    'Electronics/Computers & Tablets/iPad/Tablet/eBook Readers',
    'Electronics/Computers & Tablets/Laptops & Netbooks',
    'Electronics/Computers & Tablets/iPad/Tablet/eBook Access'
    ]

electronics_camera_list = [
    'Electronics/Cameras & Photography/Camera & Photo Accessories',
    'Electronics/Cameras & Photography/Digital Cameras'
    ]

bag_list = [
    "Women/Women's Handbags/Shoulder Bag",
    "Women/Women's Handbags/Messenger & Crossbody",
    "Women/Women's Handbags/Backpack Style",
    "Women/Women's Handbags/Totes & Shoppers",
    'Vintage & Collectibles/Bags and Purses/Luggage',
    "Vintage & Collectibles/Bags and Purses/Purse"
    ]

jewelry_list = [
    'Women/Jewelry/Rings',
    "Women/Women's Accessories/Watches",
    'Women/Jewelry/Necklaces'
    ]

make_up_list = [
    'Beauty/Makeup/Lips',
    'Beauty/Makeup/Makeup Sets',
    'Beauty/Skin Care/Face',
    'Beauty/Makeup/Makeup Palettes'
    ]

kids_list = [
    'Kids/Toys/Electronics for Kids',
    'Kids/Boys 0-24 Mos/One-Pieces',
    'Kids/Girls 2T-5T/Coats & Jackets',
    'Kids/Girls (4+)/Shoes',
    'Kids/Diapering/Wipes & Holders',
    'Kids/Girls 0-24 Mos/Shoes',
    'Kids/Toys/Learning & Education',
    'Kids/Nursery/Bedding',
    'Kids/Girls (4+)/Coats & Jackets',
    'Kids/Boys 2T-5T/Shoes',
    'Kids/Girls (4+)/Tops & T-Shirts',
    'Kids/Toys/Arts & Crafts'
    ]


other_list = list(concat_data["category_name"][~concat_data["category_name"].isin(
    jewelry_list+
    bag_list+
    electronics_camera_list+
    electronics_phone_list+
    make_up_list+
    kids_list
    )].dropna().values)



In [5]:
other_list = list(set(other_list))

### importance_listの取得

In [6]:
def importance_list_calc(train_data,test_data,category_list,percent):
    #データの取得
    train_data = train_data[train_data["category_name"].isin(category_list)]
    test_data = test_data[test_data["category_name"].isin(category_list)]
    
    #再index
    train_data = pd.concat([train_data,test_data],axis=0)
    train_data = train_data.reset_index(drop = True)
    
    #価格順に並べ替え
    df_ascend = train_data.sort_values(by=["price"], ascending=False)
    #高級なアイテムのリストを取得
    high_price_list = df_ascend[:round(len(train_data)*percent)]
    high_price_list = high_price_list.reset_index(drop=True)
    
    #高級なアイテムのpriceを取得
    high_price_list_price = high_price_list["price"].fillna("NoData")
    
    #高級なアイテムのnameを取得
    high_price_list_name = high_price_list["name"].fillna("NoData")
    
    #NLP処理によりnameをダミー化する
    name_list = []
    for i in range(len(high_price_list_name)):
        text = high_price_list_name[i]
        name_list.append(text)
    
    replace_name_list = []
    for i in range(len(high_price_list_name)):
        text = name_list[i].replace('/', ' ')
        replace_name_list.append(text)
    
    lines = replace_name_list
    is_noun = lambda pos: pos[:2] == 'NN'

    tokenized_list = []
    for i in range(len(replace_name_list)):
        tokenized = nltk.word_tokenize(lines[i])
        nouns = [word for (word, pos) in nltk.pos_tag(tokenized)] #if is_noun(pos)]
        tokenized_list.append(nouns)

    tokenized_join = []
    for i in range(len(replace_name_list)):
        a = ','.join(tokenized_list[i])
        a = a.replace(","," ")
        tokenized_join.append(a)
    t = Tokenizer()
    t.fit_on_texts(tokenized_join)
    
    matrix = t.texts_to_matrix(tokenized_join, mode='count')
    df = pd.DataFrame(matrix)
    df = df.iloc[:,1:]
    df.columns = t.word_index
    df = df.rename(columns={"shipping":"ship-ing"})
    
    #x,yを取得
    train_data_x = df
    train_data_y = high_price_list_price
    
    #xgboost
    feature_names = list(train_data_x)
    dtrain = xgb.DMatrix(train_data_x.as_matrix(),label=train_data_y.tolist(), feature_names = feature_names)
    df_x_train = xgb.DMatrix(train_data_x)
    df_y_train = xgb.DMatrix(train_data_y)
    
    mod = xgb.XGBRegressor(learning_rate=0.1,max_depth=10,n_estimators=1000)
    mod.fit(train_data_x, train_data_y)
    
    importance = mod.booster().get_score(importance_type='weight')
    importance_list = list(importance.keys())
    
    return importance_list
    

### new_importance_list

In [27]:
def new_importance_list_calc(train_data,test_data,category_list):
    #high priceのnameの単語の抽出
    #データの取得
    train_data = train_data[train_data["category_name"].isin(category_list)]
    percent = 0.02
    #価格順に並べ替え
    df_ascend = train_data.sort_values(by=["price"], ascending=False)
    #高級なアイテムのリストを取得
    high_price_list = df_ascend[:round(len(train_data)*percent)]
    high_price_list = high_price_list.reset_index(drop=True)
    
    #高級なアイテムのpriceを取得
    high_price_list_price = high_price_list["price"].fillna("NoData")
    
    #高級なアイテムのnameを取得
    high_price_list_name = high_price_list["name"].fillna("NoData")

    tokenized_list = []
    for i in range(len(high_price_list_name)):
        tokenized = nltk.word_tokenize(high_price_list_name[i])
        nouns = [word for (word, pos) in nltk.pos_tag(tokenized)]
        tokenized_list.append(nouns)

    tokenized_join = []
    for i in range(len(high_price_list_name)):
        a = ','.join(tokenized_list[i])
        a = a.replace(","," ")
        tokenized_join.append(a)
    
    t = Tokenizer()
    t.fit_on_texts(tokenized_join)
    list_high_price = list(t.word_index.keys())
    df_high_price = pd.DataFrame(list(t.word_index.keys()))

    #normal_listの作成
    percent = 0.98
    #価格順に並べ替え
    df_ascend = train_data.sort_values(by=["price"], ascending=True)
    #普通なアイテムのリストを取得
    normal_price_list = df_ascend[:round(len(train_data)*percent)]
    normal_price_list = normal_price_list.reset_index(drop=True)
    
    #普通なアイテムのpriceを取得
    normal_price_list_price = normal_price_list["price"].fillna("NoData")
    
    #普通なアイテムのnameを取得
    normal_price_list_name = normal_price_list["name"].fillna("NoData")

    tokenized_list = []
    for i in range(len(normal_price_list_name)):
        tokenized = nltk.word_tokenize(normal_price_list_name[i])
        nouns = [word for (word, pos) in nltk.pos_tag(tokenized)]
        tokenized_list.append(nouns)

    tokenized_join = []
    for i in range(len(normal_price_list_name)):
        a = ','.join(tokenized_list[i])
        a = a.replace(","," ")
        tokenized_join.append(a)
    
    t = Tokenizer()
    t.fit_on_texts(tokenized_join)
    list_normal_price = list(t.word_index.keys())
    df_normal_price = pd.DataFrame(list(t.word_index.keys()))
    
    high_list = df_high_price[0].fillna("NoData")
    new_important_list = list(high_list[~high_list.isin(list_normal_price)])
    
    return new_important_list

In [14]:
new_bag = new_importance_list_calc(train_data_origin,test_data_origin,bag_list)
list(new_bag)

['bloomsbury',
 'rb',
 'drew',
 'hampstead',
 'palermo',
 'montorgueil',
 'guarenteed',
 'maxi',
 'girolata',
 'matelassé',
 '✨price',
 'drop✨ysl',
 'bar',
 'mabillon',
 'neonoe',
 'elton',
 'damien',
 'alchester',
 'turenne',
 'pivoine',
 'mazarine',
 'trevi',
 '‼️sale‼️',
 'sully',
 'kristy',
 'cecilia',
 'moira17',
 'delightfu',
 'shopria',
 'galleria',
 'boston60',
 'siracusa',
 'ashby',
 'marguerite',
 'tiecountry',
 '36',
 'caitibelle',
 'portafoglio',
 'thames',
 'bando',
 'macken',
 'sale⚡️louis',
 'alizarin',
 'trolley',
 'retiro',
 'posted',
 'odeon',
 "le'roya",
 'hayes',
 'safiano',
 'steamboat',
 'willie',
 'horseshoe',
 'dainty',
 'marks',
 'm21011',
 'shaggy',
 'sretsis',
 'diamon',
 'dyed',
 'galiera',
 'salisbury',
 'luco',
 'kings',
 'complete']

In [ ]:
list_high_price = list(t.word_index.keys())
df_high_price = pd.DataFrame(list(t.word_index.keys()))

In [ ]:
#normal priceのnameの単語の抽出
#データの取得
train_data = train_data_origin
test_data = test_data_origin
percent = 0.99
#価格順に並べ替え
df_ascend = train_data.sort_values(by=["price"], ascending=True)
#普通なアイテムのリストを取得
normal_price_list = df_ascend[:round(len(train_data)*percent)]
normal_price_list = normal_price_list.reset_index(drop=True)
    
#普通なアイテムのpriceを取得
normal_price_list_price = normal_price_list["price"].fillna("NoData")
    
#普通なアイテムのnameを取得
normal_price_list_name = normal_price_list["name"].fillna("NoData")

normal_price_list_name

tokenized_list = []
for i in range(len(normal_price_list_name)):
    tokenized = nltk.word_tokenize(normal_price_list_name[i])
    nouns = [word for (word, pos) in nltk.pos_tag(tokenized)]
    tokenized_list.append(nouns)

tokenized_join = []
for i in range(len(normal_price_list_name)):
    a = ','.join(tokenized_list[i])
    a = a.replace(","," ")
    tokenized_join.append(a)
    
t = Tokenizer()
t.fit_on_texts(tokenized_join)
    
pd_normal_price = pd.DataFrame(normal_price_list_name)

matrix = t.texts_to_matrix(tokenized_join, mode='count')
t.word_index.keys()

In [ ]:
list_normal_price = list(t.word_index.keys())
df_normal_price = pd.DataFrame(list(t.word_index.keys()))

In [ ]:
len(df_normal_price)

In [ ]:
train_data_brand_name = df_high_price[0].fillna("NoData")
train_data_brand_name[~train_data_brand_name.isin(list_normal_price)]

In [ ]:
df_high_price[0].fillna("NoData")

### train_data_xの作成

In [131]:
def make_train_data(train_data,test_data,category_list,importance_list):
    #データの取得
    train_data = train_data[train_data["category_name"].isin(category_list)]
    test_data = test_data[test_data["category_name"].isin(category_list)]
    #再index
    train_data = pd.concat([train_data,test_data],axis=0)
    train_data = train_data.reset_index(drop = True)
    #並び替え
    df_ascend = train_data.sort_values(by=["price"], ascending=False)
    #nameのダミー
    name_list = []
    for i in range(len(train_data["name"])):
        text = train_data["name"][i]
        name_list.append(text)
    
    replace_name_list = []
    for i in range(len(train_data["name"])):
        text = name_list[i].replace('/', ' ')
        replace_name_list.append(text)
    
    lines = replace_name_list
    is_noun = lambda pos: pos[:2] == 'NN'

    tokenized_list = []
    for i in range(len(replace_name_list)):
        tokenized = nltk.word_tokenize(lines[i])
        nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if word in importance_list ]
        tokenized_list.append(nouns)

    tokenized_join = []
    for i in range(len(replace_name_list)):
        a = ','.join(tokenized_list[i])
        a = a.replace(","," ")
        tokenized_join.append(a)
    t = Tokenizer()
    t.fit_on_texts(tokenized_join)
    
    matrix = t.texts_to_matrix(tokenized_join, mode='count')
    name_dummies = pd.DataFrame(matrix)
    name_dummies = name_dummies.iloc[:,1:]
    name_dummies.columns = t.word_index
    name_dummies = name_dummies.rename(columns={"shipping":"ship-ing"})
    
    #brand_nameのダミー
    brand_list = df_ascend["brand_name"][0:round(len(train_data)*0.1)]
    brand_list_name = brand_list.values
    train_data_brand_name = train_data["brand_name"].fillna("NoData")
    train_data_brand_name[~train_data_brand_name.isin(brand_list)] =  "others-brand"
    train_data_brand_name[train_data_brand_name.isin(brand_list)] = train_data_brand_name[train_data_brand_name.isin(brand_list)] + "-brand"
    brand_name_dummies = pd.get_dummies(train_data_brand_name)
    brand_name_dummies = pd.DataFrame(brand_name_dummies)
    
    #category_nameのダミー
    category_list = df_ascend["category_name"][0:len(train_data)]
    category_list_name = category_list.values
    train_data_category_name = train_data["category_name"].fillna("NoData")
    category_name_dummies = pd.get_dummies(train_data_category_name)
    category_name_dummies = pd.DataFrame(category_name_dummies)
    
    #train_data_y
    train_data_y = train_data["price"].dropna()
    
    #concat
    train_id = train_data["train_id"]
    test_id = train_data["test_id"]
    test_id = test_id.dropna().values
    
    train_data.drop(["name","category_name","brand_name","price","item_description"],axis=1,inplace=True)
    item_dummy = pd.get_dummies(train_data["item_condition_id"])
    item_dummy.columns = ["1.0","2.0","3.0","4.0","5.0"]
    
    train_data_x = pd.concat([train_data["shipping"],train_data["train_id"],train_data["test_id"],item_dummy,name_dummies,brand_name_dummies,category_name_dummies],axis=1)
    
    train_data_x.drop("test_id",axis=1,inplace=True)
    train_data_x = train_data_x.dropna()
    train_data_x.drop("train_id",axis=1,inplace=True)
    
    test_data_x = pd.concat([train_data["shipping"],train_data["train_id"],train_data["test_id"],item_dummy,name_dummies,brand_name_dummies,category_name_dummies],axis=1)
    test_data_x.drop("train_id",axis=1,inplace=True)
    test_data_x = test_data_x.dropna()
    test_data_x.drop("test_id",axis=1,inplace=True)
    test_data_x.index = test_id
    
    return train_data_x,train_data_y,test_data_x,test_id

### normal_dataのtrain_data_x

In [16]:
def make_train_data_normal(train_data,test_data,category_list):
    #データの取得
    train_data = train_data[train_data["category_name"].isin(category_list)]
    test_data = test_data[test_data["category_name"].isin(category_list)]
    #再index
    train_data = pd.concat([train_data,test_data],axis=0)
    train_data = train_data.reset_index(drop = True)
    #並び替え
    df_ascend = train_data.sort_values(by=["price"], ascending=False)
    
    #brand_nameのダミー
    brand_list = df_ascend["brand_name"][0:round(len(train_data)*0.01)]
    brand_list_name = brand_list.values
    train_data_brand_name = train_data["brand_name"].fillna("NoData") 
    train_data_brand_name[~train_data_brand_name.isin(brand_list)] =  "others-brand"
    brand_name_dummies = pd.get_dummies(train_data_brand_name)
    brand_name_dummies = pd.DataFrame(brand_name_dummies)
    
    #category_nameのダミー
    category_list = df_ascend["category_name"][0:len(train_data)]
    category_list_name = category_list.values
    train_data_category_name = train_data["category_name"].fillna("NoData")    
    category_name_dummies = pd.get_dummies(train_data_category_name)
    category_name_dummies = pd.DataFrame(category_name_dummies)
    
    #train_data_y
    train_data_y = train_data["price"].dropna()
    
    #concat
    train_id = train_data["train_id"]
    test_id = train_data["test_id"]
    test_id = test_id.dropna().values
    
    train_data.drop(["name","category_name","brand_name","price","item_description"],axis=1,inplace=True)
    item_dummy = pd.get_dummies(train_data["item_condition_id"])
    item_dummy.columns = ["1.0","2.0","3.0","4.0","5.0"]
    
    train_data_x = pd.concat([train_data["shipping"],train_data["train_id"],train_data["test_id"],item_dummy,brand_name_dummies,category_name_dummies],axis=1)
    
    train_data_x.drop("test_id",axis=1,inplace=True)
    train_data_x = train_data_x.dropna()
    train_data_x.drop("train_id",axis=1,inplace=True)
    
    test_data_x = pd.concat([train_data["shipping"],train_data["train_id"],train_data["test_id"],item_dummy,brand_name_dummies,category_name_dummies],axis=1)
    test_data_x.drop("train_id",axis=1,inplace=True)
    test_data_x = test_data_x.dropna()
    test_data_x.drop("test_id",axis=1,inplace=True)
    test_data_x.index = test_id
    
    return train_data_x,train_data_y,test_data_x,test_id

### category_nullのtrain_data_x

In [8]:
def make_train_data_null_category(train_data,test_data):
    #データの取得
    train_data = train_data[train_data["category_name"].isnull()]
    test_data = test_data[test_data["category_name"].isnull()]
    #再index
    train_data = pd.concat([train_data,test_data],axis=0)
    train_data = train_data.reset_index(drop = True)
    #並び替え
    df_ascend = train_data.sort_values(by=["price"], ascending=False)
    
    #brand_nameのダミー
    brand_list = df_ascend["brand_name"][0:round(len(train_data)*0.1)]
    brand_list_name = brand_list.values
    train_data_brand_name = train_data["brand_name"].fillna("NoData")
    train_data_brand_name[~train_data_brand_name.isin(brand_list)] =  "others-brand"
    brand_name_dummies = pd.get_dummies(train_data_brand_name)
    brand_name_dummies = pd.DataFrame(brand_name_dummies)
    
    
    #category_nameのダミー=null
    
    #train_data_y
    train_data_y = train_data["price"].dropna()
    
    #concat
    train_id = train_data["train_id"]
    test_id = train_data["test_id"]
    test_id = test_id.dropna().values
    
    train_data.drop(["name","category_name","brand_name","price","item_description"],axis=1,inplace=True)
    item_dummy = pd.get_dummies(train_data["item_condition_id"])
    item_dummy.columns = ["1.0","2.0","3.0","4.0","5.0"]
    
    train_data_x = pd.concat([train_data["shipping"],train_data["train_id"],train_data["test_id"],item_dummy,brand_name_dummies],axis=1)
    
    train_data_x.drop("test_id",axis=1,inplace=True)
    train_data_x = train_data_x.dropna()
    train_data_x.drop("train_id",axis=1,inplace=True)
    
    test_data_x = pd.concat([train_data["shipping"],train_data["train_id"],train_data["test_id"],item_dummy,brand_name_dummies],axis=1)
    test_data_x.drop("train_id",axis=1,inplace=True)
    test_data_x = test_data_x.dropna()
    test_data_x.drop("test_id",axis=1,inplace=True)
    test_data_x.index = test_id
    
    return train_data_x,train_data_y,test_data_x,test_id

### Xgboostによる学習と予測データのoutput

In [9]:
def train_and_output(train_data_x,train_data_y,test_data_x):
    #train
    feature_names = list(train_data_x)
    dtrain = xgb.DMatrix(train_data_x.as_matrix(),label=train_data_y.tolist(), feature_names = feature_names)
    df_x_train = xgb.DMatrix(train_data_x)
    df_y_train = xgb.DMatrix(train_data_y)
    mod = xgb.XGBRegressor(learning_rate=0.1,max_depth=10,n_estimators=10)
    mod.fit(train_data_x, train_data_y)
    
    #predict
    y_test_pred = mod.predict(test_data_x)
    df_y_test = pd.DataFrame()
    df_y_test_pred = pd.DataFrame(y_test_pred)
    df_y_test_pred.index = test_data_x.index
    
    df_output = pd.DataFrame()
    df_output["test_id"] = df_y_test_pred.index
    df_output["price"] = df_y_test_pred.values
    
    return df_output

### 全行程の統合

In [10]:
#null_category_price
df_result_null = pd.DataFrame()
train_data = train_data_origin
test_data = test_data_origin
make_data = make_train_data_null_category(train_data,test_data)
df_output = train_and_output(make_data[0],make_data[1],make_data[2])
df_result_null = pd.concat([df_result_null,df_output],axis=0)

In [113]:
j = len(other_list)
unit = 70

for k in range(50):
    if k*unit > j:
        break

for i in range(19):
    s = unit*i
    print(unit*i,s+unit)
j,k

0 70
70 140
140 210
210 280
280 350
350 420
420 490
490 560
560 630
630 700
700 770
770 840
840 910
910 980
980 1050
1050 1120
1120 1190
1190 1260
1260 1330


(1278, 19)

In [114]:
df_result_normal = pd.DataFrame()

j = len(other_list)
unit = 70

for k in range(50):
    if k*unit > j:
        break

for i in range(k):
    s = unit*i
    category = other_list[unit*i:s+unit]
#normal_price
    train_data = train_data_origin
    test_data = test_data_origin
    make_data = make_train_data_normal(train_data,test_data,category)
    df_output = train_and_output(make_data[0],make_data[1],make_data[2])
    df_result_normal = pd.concat([df_result_normal,df_output],axis=0)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [71]:
train_data = train_data_origin
test_data = test_data_origin
percent = 0.1
category = jewelry_list
out = importance_list_calc(train_data,test_data,category,percent)
out

['✳️bundle',
 'diamond',
 'h0682',
 'apple',
 'wheaton',
 '1timedeals',
 'yurman',
 'jojo',
 'harlow',
 'michele',
 'statement',
 'inspired',
 'j',
 'quatrefoil',
 'petite',
 'ring',
 'david',
 '2',
 'bundle',
 'milanese',
 'goyard',
 'face',
 'fossil',
 'i',
 'plus',
 'marquise',
 'set',
 'soldered',
 'emerald',
 '10k',
 'necklace',
 'watch',
 'bridal',
 'gold',
 '5',
 'chronograph',
 'engagement',
 'silver',
 'hold',
 'band',
 'urban',
 'new',
 'tcw',
 'tahitian',
 'not',
 'wedding',
 'francaise',
 'originally',
 '16',
 'coquette',
 'rare',
 'smart',
 '17',
 'sony',
 'vs2',
 'albion',
 'sterling',
 '4',
 'levian',
 'authentic',
 'for',
 'piece',
 '14k',
 'carat',
 'cut',
 'sale',
 '3',
 "'s",
 'chain',
 'yellow',
 'deco',
 '6',
 'auth',
 'ii',
 'two',
 '18',
 '18k',
 'tiffany',
 'picasso',
 'pandora',
 'neil',
 '1',
 'reserved',
 'in',
 'starburst',
 'sport',
 '22k',
 'harlie',
 'mint',
 'women',
 'morganite',
 'hoops',
 'topaz',
 'red',
 'and',
 'scott',
 'nat',
 'havana',
 'tedi',


### importance_listをoutput

In [13]:
def importance_list_output(train_data,test_data,category,percent):
    importance_merge = pd.DataFrame()
    for category in category_list:
        importance_data = importance_list_calc(train_data,test_data,category,percent)
        df_importance_data = pd.DataFrame(importance_data)
        importance_merge = pd.concat([importance_merge,df_importance_data],axis=1)
    
    return importance_merge
    

In [28]:
def new_importance_list_output(train_data,test_data,category_list):
    importance_merge = pd.DataFrame()
    for category in category_list:
        importance_data = new_importance_list_calc(train_data,test_data,category)
        df_importance_data = pd.DataFrame(importance_data)
        importance_merge = pd.concat([importance_merge,df_importance_data],axis=1)
    
    return importance_merge
    

In [29]:
train_data = train_data_origin
test_data = test_data_origin
category_list = jewelry_list,electronics_camera_list,bag_list,electronics_phone_list,make_up_list,kids_list
importance_out = new_importance_list_output(train_data,test_data,category_list)
importance_out

,0,0,0,0,0,0
0,jojo,70d,bloomsbury,512,tria,pengualas
1,neil,karma,☆nwt☆,512gb,salazars,bearakeet
2,tedi,7d,brea,5ghz,lashboost,owlicorn
3,✳️bundle,6d,marcie,specs,restock,hottest
4,vs2,d5500,rb,”,dube,poponngay
5,h0682,m3,drew,reducing,cells,pengula
6,✳️watches,d7100,hampstead,pro15in,tapple,swivel
7,1timedeals,NaN,insignia,1796,3some,nes
8,jojo✳️,NaN,palermo,ipadpro12,group,cameo
9,soldered,NaN,montorgueil,wintawonderland,regimens,bonus


In [142]:
importance_out = importance_out[:100]
importance_out

,jewelry_list,electronics_camera_list,bag_list,electronics_phone_list,make_up_list,kids_list
0,✳️bundle,70d,vuitton,pro,regimens,hatchimals
1,diamond,t6i,chanel,7,bundle,hatchimal
2,h0682,phantom,mcm,macbook,blushsense,janome
3,apple,sony,mm,6s,regimen,dockatot
4,wheaton,4,saint,products,500,cricut
5,1timedeals,hero3,soho,gaming,fields,brother
6,yurman,nex,kors,book,280,9300hailey
7,jojo,dji,kate,plus,quit,boost
8,harlow,canon,ysl,s7,palette,velvet
9,michele,t1i,prada,pixel,hand,poponngay


In [32]:
importance_out.columns = ["jewelry_list","electronics_camera_list","bag_list","electronics_phone_list","make_up_list","kids_list"]

In [33]:
importance_out.to_csv("/Users/ishimotoyoshitake/Desktop/kaggle_datasets/mercari/new_importance_list.csv",index=False)

In [126]:
import codecs
with codecs.open("/Users/ishimotoyoshitake/Desktop/kaggle_datasets/mercari/importance_list.csv", "r", "Shift-JIS", "ignore") as file:
    read_importance = pd.read_table(file, delimiter=",")

In [127]:
a = "jewelry_list"
list(read_importance[a].dropna().values)

['??bundle',
 'diamond',
 'h0682',
 'apple',
 'wheaton',
 '1timedeals',
 'yurman',
 'jojo',
 'harlow',
 'michele',
 'statement',
 'inspired',
 'j',
 'quatrefoil',
 'petite',
 'ring',
 'david',
 '2',
 'bundle',
 'milanese',
 'goyard',
 'face',
 'fossil',
 'i',
 'plus',
 'marquise',
 'set',
 'soldered',
 'emerald',
 '10k',
 'necklace',
 'watch',
 'bridal',
 'gold',
 '5',
 'chronograph',
 'engagement',
 'silver',
 'hold',
 'band',
 'urban',
 'new',
 'tcw',
 'tahitian',
 'not',
 'wedding',
 'francaise',
 'originally',
 '16',
 'coquette',
 'rare',
 'smart',
 '17',
 'sony',
 'vs2',
 'albion',
 'sterling',
 '4',
 'levian',
 'authentic',
 'for',
 'piece',
 '14k',
 'carat',
 'cut',
 'sale',
 '3',
 "'s",
 'chain',
 'yellow',
 'deco',
 '6',
 'auth',
 'ii',
 'two',
 '18',
 '18k',
 'tiffany',
 'picasso',
 'pandora',
 'neil',
 '1',
 'reserved',
 'in',
 'starburst',
 'sport',
 '22k',
 'harlie',
 'mint',
 'women',
 'morganite',
 'hoops',
 'topaz',
 'red',
 'and',
 'scott',
 'nat',
 'havana',
 'tedi',


In [132]:
category_list_high = {"jewelry_list":jewelry_list,
                      "electronics_camera_list":electronics_camera_list,
                      "bag_list":bag_list,
                      "electronics_phone_list":electronics_phone_list,
                      "make_up_list":make_up_list,
                      "kids_list":kids_list}

#high_price
df_result_high = pd.DataFrame()
for (category_values,category_keys) in zip(category_list_high.values(),category_list_high.keys()):
    train_data = train_data_origin
    test_data = test_data_origin
    percent = 0.1
    importance_list = list(read_importance[category_keys].dropna().values)
    make_data = make_train_data(train_data,test_data,category_values,importance_list)
    df_output = train_and_output(make_data[0],make_data[1],make_data[2])
    df_result_high = pd.concat([df_result_high,df_output],axis=0)
    print(category_values)

['Women/Jewelry/Rings', "Women/Women's Accessories/Watches", 'Women/Jewelry/Necklaces']
['Electronics/Cameras & Photography/Camera & Photo Accessories', 'Electronics/Cameras & Photography/Digital Cameras']
["Women/Women's Handbags/Shoulder Bag", "Women/Women's Handbags/Messenger & Crossbody", "Women/Women's Handbags/Backpack Style", "Women/Women's Handbags/Totes & Shoppers", 'Vintage & Collectibles/Bags and Purses/Luggage', 'Vintage & Collectibles/Bags and Purses/Purse']
['Electronics/Cell Phones & Accessories/Cell Phone Accessories', 'Electronics/Cell Phones & Accessories/Cell Phones & Smartphones', 'Electronics/Computers & Tablets/iPad/Tablet/eBook Readers', 'Electronics/Computers & Tablets/Laptops & Netbooks', 'Electronics/Computers & Tablets/iPad/Tablet/eBook Access']
['Beauty/Makeup/Lips', 'Beauty/Makeup/Makeup Sets', 'Beauty/Skin Care/Face', 'Beauty/Makeup/Makeup Palettes']
['Kids/Toys/Electronics for Kids', 'Kids/Boys 0-24 Mos/One-Pieces', 'Kids/Girls 2T-5T/Coats & Jackets', 'Ki

In [143]:
make_data[1]

0          8.0
1         36.0
2         14.0
3        150.0
4         36.0
5          7.0
6         12.0
7         30.0
8         22.0
9          9.0
10        13.0
11        21.0
12        24.0
13        12.0
14        20.0
15        13.0
16        12.0
17         8.0
18        36.0
19        13.0
20        18.0
21        10.0
22        13.0
23        52.0
24         5.0
25        24.0
26        10.0
27         4.0
28         8.0
29        22.0
         ...  
36496      9.0
36497     16.0
36498     15.0
36499     12.0
36500     19.0
36501     36.0
36502     81.0
36503     21.0
36504     14.0
36505     21.0
36506     36.0
36507     14.0
36508     31.0
36509     11.0
36510     51.0
36511     10.0
36512    124.0
36513      9.0
36514     15.0
36515      6.0
36516      5.0
36517     30.0
36518     33.0
36519     26.0
36520     36.0
36521    116.0
36522      8.0
36523     26.0
36524     22.0
36525     18.0
Name: price, Length: 36526, dtype: float64

In [135]:
df_result = pd.concat([df_result_high,df_result_normal,df_result_null],axis=0)

In [139]:
df_result_high.sort_values(by=["price"], ascending=False)

,test_id,price
10127,380827.0,834.700867
18072,680586.0,834.700867
8960,337923.0,620.003296
3076,115159.0,620.003296
391,15140.0,620.003296
9508,358609.0,620.003296
10978,412817.0,620.003296
16831,632927.0,602.193970
5715,214940.0,602.193970
12625,472462.0,562.055481


In [136]:
df_result.sort_values(by=["test_id"], ascending=True)

,test_id,price
0,0.0,7.850902
0,1.0,6.805652
0,2.0,48.896652
0,3.0,12.269313
0,4.0,11.944784
0,5.0,7.376049
1,6.0,20.502235
2,7.0,15.557863
0,8.0,16.409700
1,9.0,9.054135


In [35]:
df_sort = df_result.sort_values(by=["test_id"], ascending=True)

In [41]:
df_sort = df_sort.assign(test_id = df_sort["test_id"].astype(np.int32))

In [42]:
df_sort["test_id"].dtype

dtype('int32')

In [ ]:
df_sort[['test_id','price']].to_csv("/Users/ishimotoyoshitake/Desktop/kaggle_datasets/mercari/xgboost_submission.csv",index=False)

In [ ]:
df_sort[30:100]

In [ ]:
for i in range(1000000):
    a = df_sort["test_id"].values[i+1] - df_sort["test_id"].values[i]
    if a != 1.0:
        print(i)

In [ ]:
df_sort["test_id"].values[5] - df_sort["test_id"].values[4]

In [ ]:
test_data_origin[52:56]

In [ ]:
if "Vintage & Collectibles/Other/Other" in other_list:
    print("yes")